# Treatment Data Preprocessing
---

Reading and preprocessing treatment data of the eICU dataset from MIT with the data from over 139k patients collected in the US.

This notebook addresses the preprocessing of the following eICU tables:
* admissionDrug
* infusionDrug
* medication
* treatment
* intakeOutput

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import yaml                                # Save and load YAML files

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir("../../../..")

# Path to the CSV dataset files
data_path = 'Datasets/Thesis/eICU/uncompressed/'

# Path to the code files
project_path = 'GitHub/eICU-mortality-prediction/'

In [ ]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
import data_utils as du                    # Data science and machine learning relevant methods

Set the random seed for reproducibility

In [ ]:
du.set_random_seed(42)

## Initialize variables

In [ ]:
cat_feat = []                              # List of categorical features
cat_embed_feat = []                        # List of categorical features that will be embedded
cat_embed_feat_enum = dict()               # Dictionary of the enumerations of the categorical features that will be embedded

## Infusion drug data

### Read the data

In [ ]:
infdrug_df = pd.read_csv(f'{data_path}original/infusionDrug.csv')
infdrug_df.head()

In [ ]:
len(infdrug_df)

In [ ]:
infdrug_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
infdrug_df.describe().transpose()

In [ ]:
infdrug_df.columns

In [ ]:
infdrug_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(infdrug_df)

### Remove unneeded features

Besides removing the row ID `infusiondrugid`, I'm also removing `infusionrate`, `volumeoffluid` and `drugamount` as they seem redundant with `drugrate` although with a lot more missing values.

In [ ]:
infdrug_df = infdrug_df.drop(['infusiondrugid', 'infusionrate', 'volumeoffluid', 'drugamount'], axis=1)
infdrug_df.head()

### Remove string drug rate values

In [ ]:
infdrug_df[infdrug_df.drugrate.map(du.utils.is_definitely_string)].head()

In [ ]:
infdrug_df[infdrug_df.drugrate.map(du.utils.is_definitely_string)].drugrate.value_counts()

In [ ]:
infdrug_df.drugrate = infdrug_df.drugrate.map(lambda x: np.nan if du.utils.is_definitely_string(x) else x)
infdrug_df.head()

In [ ]:
infdrug_df.patientunitstayid = infdrug_df.patientunitstayid.astype(int)
infdrug_df.infusionoffset = infdrug_df.infusionoffset.astype(int)
infdrug_df.drugname = infdrug_df.drugname.astype(str)
infdrug_df.drugrate = infdrug_df.drugrate.astype(float)
infdrug_df.patientweight = infdrug_df.patientweight.astype(float)
infdrug_df.head()

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['drugname']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [infdrug_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
infdrug_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Prepare for embedding, i.e. enumerate categories
    infdrug_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(infdrug_df, feature, nan_value=0)

In [ ]:
infdrug_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
infdrug_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_inf_drug.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
infdrug_df = infdrug_df.rename(columns={'infusionoffset': 'ts'})
infdrug_df.head()

Standardize drug names:

In [ ]:
infdrug_df = du.data_processing.clean_categories_naming(infdrug_df, 'drugname')
infdrug_df.head()

Remove duplicate rows:

In [ ]:
len(infdrug_df)

In [ ]:
infdrug_df = infdrug_df.drop_duplicates()
infdrug_df.head()

In [ ]:
len(infdrug_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
infdrug_df = infdrug_df.sort_values('ts')
infdrug_df.head(6)

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
infdrug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drugname', n=5).head()

In [ ]:
infdrug_df[infdrug_df.patientunitstayid == 1785711].head(20)

We can see that there are up to 17 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, as we shouldn't mix absolute values of drug rates from different drugs, we better normalize it first.

### Normalize data

In [ ]:
infdrug_df_norm = du.data_processing.normalize_data(infdrug_df,
                                                 columns_to_normalize=['patientweight'],
                                                 columns_to_normalize_cat=[('drugname', 'drugrate')])
infdrug_df_norm.head()

In [ ]:
infdrug_df_norm.patientweight.value_counts()

### Join rows that have the same IDs

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
infdrug_df_norm, pd = du.utils.convert_dataframe(infdrug_df_norm, to='pandas')

In [ ]:
type(infdrug_df_norm)

In [ ]:
infdrug_df_norm = du.embedding.join_categorical_enum(infdrug_df_norm, new_cat_embed_feat, inplace=True)
infdrug_df_norm.head()

Reconvert dataframe to Modin:

In [ ]:
infdrug_df_norm, pd = du.utils.convert_dataframe(infdrug_df_norm, to='modin')

In [ ]:
type(infdrug_df_norm)

In [ ]:
infdrug_df_norm.dtypes

In [ ]:
infdrug_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drugname', n=5).head()

In [ ]:
infdrug_df_norm[infdrug_df_norm.patientunitstayid == 1785711].head(20)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Rename columns

In [ ]:
infdrug_df = infdrug_df.rename(columns={'patientweight': 'weight', 'drugname': 'infusion_drugname',
                                        'drugrate': 'infusion_drugrate'})
infdrug_df.head()

In [ ]:
infdrug_df_norm = infdrug_df_norm.rename(columns={'patientweight': 'weight', 'drugname': 'infusion_drugname',
                                                  'drugrate': 'infusion_drugrate'})
infdrug_df_norm.head()

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
infdrug_df.columns = du.data_processing.clean_naming(infdrug_df.columns)
infdrug_df_norm.columns = du.data_processing.clean_naming(infdrug_df_norm.columns)
infdrug_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
infdrug_df.to_csv(f'{data_path}cleaned/unnormalized/infusionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
infdrug_df_norm.to_csv(f'{data_path}cleaned/normalized/infusionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
infdrug_df_norm.describe().transpose()

### Join dataframes

Merge dataframes by the unit stay, `patientunitstayid`, and the timestamp, `ts`, with a tolerence for a difference of up to 30 minutes.

In [ ]:
infdrug_df = pd.read_csv(f'{data_path}cleaned/normalized/infusionDrug.csv')
infdrug_df.head()

In [ ]:
len(infdrug)

In [ ]:
infdrug.patientunitstayid.nunique()

In [ ]:
eICU_df = pd.merge_asof(eICU_df, infdrug_df, on='ts', by='patientunitstayid', direction='nearest', tolerance=30)
eICU_df.head()

## Admission drug data

### Read the data

In [ ]:
admsdrug_df = pd.read_csv(f'{data_path}original/admissionDrug.csv')
admsdrug_df.head()

In [ ]:
len(admsdrug_df)

In [ ]:
admsdrug_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
admsdrug_df.describe().transpose()

In [ ]:
admsdrug_df.columns

In [ ]:
admsdrug_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(admsdrug_df)

### Remove unneeded features

In [ ]:
admsdrug_df.drugname.value_counts()

In [ ]:
admsdrug_df.drughiclseqno.value_counts()

In [ ]:
admsdrug_df.drugnotetype.value_counts()

In [ ]:
admsdrug_df.drugdosage.value_counts()

In [ ]:
admsdrug_df.drugunit.value_counts()

In [ ]:
admsdrug_df.drugadmitfrequency.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].head(20)

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].drugunit.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].drugadmitfrequency.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugunit == ' '].drugdosage.value_counts()

Oddly, `drugunit` and `drugadmitfrequency` have several blank values. At the same time, when this happens, `drugdosage` tends to be 0 (which is also an unrealistic value). Considering that no NaNs are reported, these blanks and zeros probably represent missing values.

Besides removing irrelevant or hospital staff related data (e.g. `usertype`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
admsdrug_df = admsdrug_df[['patientunitstayid', 'drugoffset', 'drugdosage',
                           'drugunit', 'drugadmitfrequency', 'drughiclseqno']]
admsdrug_df.head()

### Fix missing values representation

Replace blank and unrealistic zero values with NaNs.

In [ ]:
admsdrug_df.drugdosage = admsdrug_df.drugdosage.replace(to_replace=0, value=np.nan)
admsdrug_df.drugunit = admsdrug_df.drugunit.replace(to_replace=' ', value=np.nan)
admsdrug_df.drugadmitfrequency = admsdrug_df.drugadmitfrequency.replace(to_replace=' ', value=np.nan)
admsdrug_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(admsdrug_df)

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

In the case of microbiology data, we're also going to embed the antibiotic `sensitivitylevel`, not because it has many categories, but because there can be several rows of data per timestamp (which would be impractical on one hot encoded data).

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['drugunit', 'drugadmitfrequency', 'drughiclseqno']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [admsdrug_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
admsdrug_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Skip the 'drughiclseqno' from enumeration encoding
    if feature == 'drughiclseqno':
        continue
    # Prepare for embedding, i.e. enumerate categories
    admsdrug_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(admsdrug_df, feature, nan_value=0)

In [ ]:
admsdrug_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
admsdrug_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_adms_drug.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
admsdrug_df = admsdrug_df.rename(columns={'drugoffset': 'ts'})
admsdrug_df.head()

Remove duplicate rows:

In [ ]:
len(admsdrug_df)

In [ ]:
admsdrug_df = admsdrug_df.drop_duplicates()
admsdrug_df.head()

In [ ]:
len(admsdrug_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
admsdrug_df = admsdrug_df.sort_values('ts')
admsdrug_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
admsdrug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
admsdrug_df[admsdrug_df.patientunitstayid == 2346930].head(10)

We can see that there are up to 48 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Normalize data

In [ ]:
admsdrug_df_norm = du.data_processing.normalize_data(admsdrug_df_norm, columns_to_normalize=False,
                                                  columns_to_normalize_cat=[(['drughiclseqno', 'drugunit'], 'drugdosage')])
admsdrug_df_norm.head()

In [ ]:
admsdrug_df_norm = admsdrug_df_norm.sort_values('ts')
admsdrug_df_norm.head()

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
admsdrug_df_norm, pd = du.utils.convert_dataframe(admsdrug_df_norm, to='pandas')

In [ ]:
type(admsdrug_df_norm)

In [ ]:
admsdrug_df_norm = du.embedding.join_categorical_enum(admsdrug_df_norm, new_cat_embed_feat, inplace=True)
admsdrug_df_norm.head()

Reconvert dataframe to Modin:

In [ ]:
admsdrug_df_norm, pd = du.utils.convert_dataframe(admsdrug_df_norm, to='modin')

In [ ]:
type(admsdrug_df_norm)

In [ ]:
admsdrug_df_norm.dtypes

In [ ]:
admsdrug_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
admsdrug_df_norm[admsdrug_df_norm.patientunitstayid == 2346930].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
admsdrug_df.columns = du.data_processing.clean_naming(admsdrug_df.columns)
admsdrug_df_norm.columns = du.data_processing.clean_naming(admsdrug_df_norm.columns)
admsdrug_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
admsdrug_df.to_csv(f'{data_path}cleaned/unnormalized/admissionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
admsdrug_df_norm.to_csv(f'{data_path}cleaned/normalized/admissionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
admsdrug_df_norm.describe().transpose()

### Join dataframes

Merge dataframes by the unit stay, `patientunitstayid`, and the timestamp, `ts`, with a tolerence for a difference of up to 30 minutes.

In [ ]:
admsdrug_df = pd.read_csv(f'{data_path}cleaned/normalized/admissionDrug.csv')
admsdrug_df.head()

In [ ]:
len(admsdrug_df)

In [ ]:
admsdrug_df.patientunitstayid.nunique()

In [ ]:
eICU_df = pd.merge_asof(eICU_df, admsdrug_df, on='ts', by='patientunitstayid', direction='nearest', tolerance=30)
eICU_df.head()

## Medication data

### Read the data

In [ ]:
med_df = pd.read_csv(f'{data_path}original/medication.csv', dtype={'loadingdose': 'object'})
med_df.head()

In [ ]:
len(med_df)

In [ ]:
med_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
med_df.describe().transpose()

In [ ]:
med_df.columns

In [ ]:
med_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Remove unneeded features

In [ ]:
med_df.drugname.value_counts()

In [ ]:
med_df.drughiclseqno.value_counts()

In [ ]:
med_df.dosage.value_counts()

In [ ]:
med_df.frequency.value_counts()

In [ ]:
med_df.drugstartoffset.value_counts()

In [ ]:
med_df[med_df.drugstartoffset == 0].head()

Besides removing less interesting data (e.g. `drugivadmixture`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
med_df = med_df[['patientunitstayid', 'drugstartoffset', 'drugstopoffset',
                 'drugordercancelled', 'dosage', 'frequency', 'drughiclseqno']]
med_df.head()

### Remove rows of which the drug has been cancelled or not specified

In [ ]:
med_df.drugordercancelled.value_counts()

In [ ]:
med_df = med_df[~((med_df.drugordercancelled == 'Yes') | (np.isnan(med_df.drughiclseqno)))]
med_df.head()

Remove the now unneeded `drugordercancelled` column:

In [ ]:
med_df = med_df.drop('drugordercancelled', axis=1)
med_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Separating units from dosage

In order to properly take into account the dosage quantities, as well as to standardize according to other tables like admission drugs, we should take the original `dosage` column and separate it to just the `drugdosage` values and the `drugunit`.

No need to create a separate `pyxis` feature, which would indicate the use of the popular automated medications manager, as the frequency embedding will have that into account.

Create dosage and unit features:

In [ ]:
med_df['drugdosage'] = np.nan
med_df['drugunit'] = np.nan
med_df.head()

Get the dosage and unit values for each row:

In [ ]:
med_df[['drugdosage', 'drugunit']] = med_df.apply(du.data_processing.set_dosage_and_units, axis=1, result_type='expand')
med_df.head()

Remove the now unneeded `dosage` column:

In [ ]:
med_df = med_df.drop('dosage', axis=1)
med_df.head()

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

In the case of microbiology data, we're also going to embed the antibiotic `sensitivitylevel`, not because it has many categories, but because there can be several rows of data per timestamp (which would be impractical on one hot encoded data).

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['drugunit', 'frequency', 'drughiclseqno']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [med_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
med_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Skip the 'drughiclseqno' from enumeration encoding
    if feature == 'drughiclseqno':
        continue
    # Prepare for embedding, i.e. enumerate categories
    med_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(med_df, feature, nan_value=0)

In [ ]:
med_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
med_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_med.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create drug stop event

Add a timestamp corresponding to when each patient stops taking each medication.

Duplicate every row, so as to create a discharge event:

In [ ]:
new_df = med_df.copy()
new_df.head()

Set the new dataframe's rows to have the drug stop timestamp, with no more information on the drug that was being used:

In [ ]:
new_df.drugstartoffset = new_df.drugstopoffset
new_df.drugunit = np.nan
new_df.drugdosage = np.nan
new_df.frequency = np.nan
new_df.drughiclseqno = np.nan
new_df.head()

Join the new rows to the remaining dataframe:

In [ ]:
med_df = med_df.append(new_df)
med_df.head()

Remove the now unneeded medication stop column:

In [ ]:
med_df = med_df.drop('drugstopoffset', axis=1)
med_df.head(6)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
med_df = med_df.rename(columns={'drugstopoffset': 'ts'})
med_df.head()

Remove duplicate rows:

In [ ]:
len(med_df)

In [ ]:
med_df = med_df.drop_duplicates()
med_df.head()

In [ ]:
len(med_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
med_df = med_df.sort_values('ts')
med_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
med_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df[med_df.patientunitstayid == 979183].head(10)

We can see that there are up to 41 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Normalize data

In [ ]:
med_df_norm = du.data_processing.normalize_data(med_df_norm, columns_to_normalize=False,
                                             columns_to_normalize_cat=[(['drughiclseqno', 'drugunit'], 'drugdosage')])
med_df_norm.head()

In [ ]:
med_df_norm = med_df_norm.sort_values('ts')
med_df_norm.head()

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

In [ ]:
list(set(med_df_norm.columns) - set(new_cat_embed_feat) - set(['patientunitstayid', 'ts']))

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
med_df_norm, pd = du.utils.convert_dataframe(med_df, to='pandas')

In [ ]:
type(med_df_norm)

In [ ]:
med_df_norm = du.embedding.join_categorical_enum(med_df_norm, new_cat_embed_feat, inplace=True)
med_df_norm.head()

Reconvert dataframe to Modin:

In [ ]:
med_df_norm, pd = du.utils.convert_dataframe(med_df, to='modin')

In [ ]:
type(med_df_norm)

In [ ]:
med_df_norm.dtypes

In [ ]:
med_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df_norm[med_df_norm.patientunitstayid == 979183].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Rename columns

In [ ]:
med_df_norm = med_df_norm.rename(columns={'frequency':'drugadmitfrequency'})
med_df_norm.head()

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
med_df.columns = du.data_processing.clean_naming(med_df.columns)
med_df_norm.columns = du.data_processing.clean_naming(med_df_norm.columns)
med_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
med_df.to_csv(f'{data_path}cleaned/unnormalized/medication.csv')

Save the dataframe after normalizing:

In [ ]:
med_df_norm.to_csv(f'{data_path}cleaned/normalized/medication.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
med_df_norm.describe().transpose()

In [ ]:
med_df.nlargest(columns='drugdosage', n=5)

Although the `drugdosage` looks good on mean (close to 0) and standard deviation (close to 1), it has very large magnitude minimum (-88.9) and maximum (174.1) values. Furthermore, these don't seem to be because of NaN values, whose groupby normalization could have been unideal. As such, it's hard to say if these are outliers or realistic values.

[TODO] Check if these very large extreme dosage values make sense and, if not, try to fix them.

### Join dataframes

Merge dataframes by the unit stay, `patientunitstayid`, and the timestamp, `ts`, with a tolerence for a difference of up to 30 minutes.

In [ ]:
med_df = pd.read_csv(f'{data_path}cleaned/normalized/medication.csv')
med_df.head()

In [ ]:
len(med_df)

In [ ]:
med_df.patientunitstayid.nunique()

In [ ]:
eICU_df = pd.merge_asof(eICU_df, med_df, on='ts', by='patientunitstayid', direction='nearest', tolerance=30)
eICU_df.head()

## Treatment data

### Read the data

In [ ]:
treat_df = pd.read_csv(f'{data_path}original/treatment.csv')
treat_df.head()

In [ ]:
len(treat_df)

In [ ]:
treat_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
treat_df.describe().transpose()

In [ ]:
treat_df.columns

In [ ]:
treat_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(treat_df)

### Remove unneeded features

Besides the usual removal of row identifier, `treatmentid`, I'm also removing `activeupondischarge`, as we don't have complete information as to when diagnosis end.

In [ ]:
treat_df = treat_df.drop(['treatmentid', 'activeupondischarge'], axis=1)
treat_df.head()

### Separate high level diagnosis

In [ ]:
treat_df.treatmentstring.value_counts()

In [ ]:
treat_df.treatmentstring.map(lambda x: x.split('|')).head()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).min()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).max()

There are always at least 3 higher level diagnosis. It could be beneficial to extract those first 3 levels to separate features, with the last one getting values until the end of the string, so as to avoid the need for the model to learn similarities that are already known.

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 3, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 4, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 5, separator='|')).value_counts()

<!-- There are always 8 levels of the notes. As the first 6 ones are essentially always the same ("notes/Progress Notes/Social History / Family History/Social History/Social History/"), it's best to just preserve the 7th one and isolate the 8th in a new feature. This way, the split provides further insight to the model on similar notes. However, it's also worth taking note that the 8th level of `notepath` seems to be identical to the feature `notevalue`. We'll look more into it later. -->

In [ ]:
treat_df['treatmenttype'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|'))
treat_df['treatmenttherapy'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|'))
treat_df['treatmentdetails'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|', till_the_end=True))
treat_df.head()

Remove the now redundant `treatmentstring` column:

In [ ]:
treat_df = treat_df.drop('treatmentstring', axis=1)
treat_df.head()

In [ ]:
treat_df.treatmenttype.value_counts()

In [ ]:
treat_df.treatmenttherapy.value_counts()

In [ ]:
treat_df.treatmentdetails.value_counts()

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['treatmenttype', 'treatmenttherapy', 'treatmentdetails']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [treat_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
treat_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Prepare for embedding, i.e. enumerate categories
    treat_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(treat_df, feature, nan_value=0)

In [ ]:
treat_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
treat_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_treat.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
treat_df = treat_df.rename(columns={'treatmentoffset': 'ts'})
treat_df.head()

Remove duplicate rows:

In [ ]:
len(treat_df)

In [ ]:
treat_df = treat_df.drop_duplicates()
treat_df.head()

In [ ]:
len(treat_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
treat_df = treat_df.sort_values('ts')
treat_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

We can see that there are up to 105 categories per set of `patientunitstayid` and `ts`. As such, we must join them.

### Join rows that have the same IDs

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='pandas')

In [ ]:
type(treat_df)

In [ ]:
treat_df = du.embedding.join_categorical_enum(treat_df, new_cat_embed_feat, inplace=True)
treat_df.head()

Reconvert dataframe to Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='modin')

In [ ]:
type(treat_df)

In [ ]:
treat_df.dtypes

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
treat_df.columns = du.data_processing.clean_naming(treat_df.columns)
treat_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
treat_df.to_csv(f'{data_path}cleaned/unnormalized/diagnosis.csv')

Save the dataframe after normalizing:

In [ ]:
treat_df.to_csv(f'{data_path}cleaned/normalized/diagnosis.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
treat_df.describe().transpose()

### Join dataframes

Merge dataframes by the unit stay, `patientunitstayid`, and the timestamp, `ts`, with a tolerence for a difference of up to 30 minutes.

In [ ]:
treat_df = pd.read_csv(f'{data_path}cleaned/normalized/diagnosis.csv')
treat_df.head()

In [ ]:
len(treat_df)

In [ ]:
treat_df.patientunitstayid.nunique()

In [ ]:
eICU_df = pd.merge_asof(eICU_df, treat_df, on='ts', by='patientunitstayid', direction='nearest', tolerance=30)
eICU_df.head()